# Single Frame Model

This script trains a single frame model based on an average image taken across the the forcasted month.

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, save_model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, Conv3D, MaxPooling2D, MaxPooling3D, LayerNormalization
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import initializers
import time
import numpy as np

## 0. Define Region

First, define a region to train a model for.

In [2]:
REGIONS = ["ES","NS","WS","EM","EE","LD","NEE","NWE","SEE","SWE","WAL","WM","YH"]

## 1. Load Data

To begin, data for a single frame mnodel must be loaded along with the expected regional rainfall values.

### 1.0 New Training Data

This section loads the new training data set.

In [3]:
training_datafile = "D:/PHD_DATA/Video_25-02-2021/prepared-data/single_train.npy"
training_rainfallfile = "D:/PHD_DATA/Video_25-02-2021/prepared-data/expected_train_standardized.npy"

In [4]:
training_videos = np.load(training_datafile)
training_rainfall = np.load(training_rainfallfile).T

In [5]:
training_videos = np.swapaxes(training_videos, 1, 2)
training_videos = np.swapaxes(training_videos, 2, 3)

In [6]:
def apply_augmentation(raw_rain, raw_vids):
    vids = np.copy(raw_vids)
    vids = vids + np.random.normal(0, 1, vids.shape)
    return np.concatenate((raw_vids, vids), axis=0), np.concatenate((raw_rain, raw_rain), axis=0)

APPLY_AUGMENTATION = False

if APPLY_AUGMENTATION:
    training_videos, training_rainfall = apply_augmentation(training_rainfall, training_videos)

In [7]:
training_videos.shape

(204, 61, 121, 2)

## 2. Model Definition

Next, a CNN model architecture is defined.

In [8]:
def model_generator(input_shape=(2, 61, 121), learning_rate=0.1):
    """ This method generates a model definition. """
    model = Sequential()
    
    # First layer
    model.add(Conv2D(64, (2, 2), input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Second layer
    model.add(Conv2D(32, (2, 2), input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Third layer
    model.add(Conv2D(16, (2, 2), input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Final Layer
    model.add(Flatten())
    model.add(Dense(1))
    model.add(Activation('relu'))
    
    # Setup training mechanism
    model.compile(
        loss="mean_squared_error",
        optimizer=Adam(learning_rate=learning_rate))
    
    return model

## 3. Model Training

Finally, training the model using the single framed data and opening a tensorboard instance with details.

In [9]:
def train_model(run_name, tensorboard, model, xdata, ydata, models_folder="D:/PHD_DATA/Video_18-01-2021/models/"):
    """ Trains the given model with the given dataset. """
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
    history = model.fit(
        xdata,
        ydata,
        batch_size=64,
        validation_split=0.3,
        callbacks=[tensorboard, es],
        epochs=150
    )
    save_model(model, models_folder + run_name + ".mdl")
    return history

In [10]:
# Flexible parameters
learning_rate = 0.001

In [11]:
final_errors = {}
# Run model for each region
for ridx, r in enumerate(REGIONS):
    final_errors[r] = {}
    # Run each model multiple times
    for i in range(0, 3):
        run_name = "MF_{}_{}".format(r, int(time.time()))
        tb = TensorBoard(log_dir="D:/PHD_DATA/Video_25-02-2021/logs/{}".format(run_name))
        model = model_generator(learning_rate=learning_rate, input_shape=training_videos.shape[1:])
        history = train_model(run_name, tb, model, training_videos, training_rainfall[:, ridx],
                              models_folder="D:/PHD_DATA/Video_25-02-2021/models/")
        final_errors[r][run_name] = history.history["val_loss"][-1]

Epoch 1/150
3/3 [==============================] - 6s 2s/step - loss: 0.0642 - val_loss: 0.0733
Epoch 2/150
3/3 [==============================] - 1s 500ms/step - loss: 0.0603 - val_loss: 0.0374
Epoch 3/150
3/3 [==============================] - 1s 406ms/step - loss: 0.0315 - val_loss: 0.0474
Epoch 4/150
3/3 [==============================] - 1s 418ms/step - loss: 0.0388 - val_loss: 0.0385
Epoch 5/150
3/3 [==============================] - 1s 276ms/step - loss: 0.0298 - val_loss: 0.0343
Epoch 6/150
3/3 [==============================] - 1s 320ms/step - loss: 0.0263 - val_loss: 0.0336
Epoch 7/150
3/3 [==============================] - 1s 397ms/step - loss: 0.0264 - val_loss: 0.0324
Epoch 8/150
3/3 [==============================] - 1s 284ms/step - loss: 0.0248 - val_loss: 0.0365
Epoch 9/150
3/3 [==============================] - 1s 400ms/step - loss: 0.0267 - val_loss: 0.0358
Epoch 10/150
3/3 [==============================] - 1s 353ms/step - loss: 0.0239 - val_loss: 0.0339
Epoch 11/150

3/3 [==============================] - 1s 388ms/step - loss: 0.0364 - val_loss: 0.0419
Epoch 12/150
3/3 [==============================] - 1s 355ms/step - loss: 0.0356 - val_loss: 0.0409
Epoch 13/150
3/3 [==============================] - 1s 305ms/step - loss: 0.0363 - val_loss: 0.0400
Epoch 14/150
3/3 [==============================] - 1s 400ms/step - loss: 0.0357 - val_loss: 0.0398
Epoch 15/150
3/3 [==============================] - 1s 335ms/step - loss: 0.0391 - val_loss: 0.0414
Epoch 16/150
3/3 [==============================] - 1s 309ms/step - loss: 0.0341 - val_loss: 0.0420
Epoch 17/150
3/3 [==============================] - 1s 355ms/step - loss: 0.0339 - val_loss: 0.0403
Epoch 18/150
3/3 [==============================] - 1s 357ms/step - loss: 0.0357 - val_loss: 0.0399
Epoch 19/150
3/3 [==============================] - 1s 335ms/step - loss: 0.0338 - val_loss: 0.0395
Epoch 20/150
3/3 [==============================] - 1s 310ms/step - loss: 0.0343 - val_loss: 0.0387
Epoch 21/150


3/3 [==============================] - 1s 357ms/step - loss: 0.0275 - val_loss: 0.0336
Epoch 18/150
3/3 [==============================] - 1s 272ms/step - loss: 0.0260 - val_loss: 0.0329
Epoch 19/150
3/3 [==============================] - 1s 357ms/step - loss: 0.0254 - val_loss: 0.0328
Epoch 20/150
3/3 [==============================] - 1s 275ms/step - loss: 0.0245 - val_loss: 0.0331
Epoch 21/150
3/3 [==============================] - 1s 403ms/step - loss: 0.0250 - val_loss: 0.0327
Epoch 22/150
3/3 [==============================] - 1s 320ms/step - loss: 0.0258 - val_loss: 0.0331
Epoch 23/150
3/3 [==============================] - 1s 352ms/step - loss: 0.0237 - val_loss: 0.0338
Epoch 24/150
3/3 [==============================] - 1s 313ms/step - loss: 0.0250 - val_loss: 0.0344
Epoch 25/150
3/3 [==============================] - 1s 391ms/step - loss: 0.0226 - val_loss: 0.0354
Epoch 26/150
3/3 [==============================] - 1s 319ms/step - loss: 0.0249 - val_loss: 0.0348
Epoch 00026: 

Epoch 1/150
3/3 [==============================] - 7s 2s/step - loss: 0.0677 - val_loss: 0.0396
Epoch 2/150
3/3 [==============================] - 1s 417ms/step - loss: 0.0347 - val_loss: 0.0399
Epoch 3/150
3/3 [==============================] - 1s 400ms/step - loss: 0.0323 - val_loss: 0.0356
Epoch 4/150
3/3 [==============================] - 1s 434ms/step - loss: 0.0282 - val_loss: 0.0325
Epoch 5/150
3/3 [==============================] - 1s 475ms/step - loss: 0.0253 - val_loss: 0.0334
Epoch 6/150
3/3 [==============================] - 1s 285ms/step - loss: 0.0255 - val_loss: 0.0343
Epoch 7/150
3/3 [==============================] - 1s 358ms/step - loss: 0.0244 - val_loss: 0.0297
Epoch 8/150
3/3 [==============================] - 1s 312ms/step - loss: 0.0237 - val_loss: 0.0273
Epoch 9/150
3/3 [==============================] - 1s 340ms/step - loss: 0.0201 - val_loss: 0.0267
Epoch 10/150
3/3 [==============================] - 1s 373ms/step - loss: 0.0176 - val_loss: 0.0287
Epoch 11/150

3/3 [==============================] - 1s 350ms/step - loss: 0.0338 - val_loss: 0.0382
Epoch 12/150
3/3 [==============================] - 1s 345ms/step - loss: 0.0324 - val_loss: 0.0361
Epoch 13/150
3/3 [==============================] - 1s 308ms/step - loss: 0.0315 - val_loss: 0.0347
Epoch 14/150
3/3 [==============================] - 1s 385ms/step - loss: 0.0312 - val_loss: 0.0353
Epoch 15/150
3/3 [==============================] - 1s 343ms/step - loss: 0.0309 - val_loss: 0.0362
Epoch 16/150
3/3 [==============================] - 1s 340ms/step - loss: 0.0302 - val_loss: 0.0379
Epoch 17/150
3/3 [==============================] - 1s 315ms/step - loss: 0.0267 - val_loss: 0.0419
Epoch 18/150
3/3 [==============================] - 1s 284ms/step - loss: 0.0288 - val_loss: 0.0419
Epoch 00018: early stopping
INFO:tensorflow:Assets written to: D:/PHD_DATA/Video_25-02-2021/models/MF_WAL_1614337747.mdl\assets
Epoch 1/150
3/3 [==============================] - 7s 2s/step - loss: 0.1485 - val_lo

In [12]:
final_errors

{'ES': {'MF_ES_1614336702': 0.030699096620082855,
  'MF_ES_1614336742': 0.030954236164689064,
  'MF_ES_1614336769': 0.02928706258535385},
 'NS': {'MF_NS_1614336799': 0.04174765571951866,
  'MF_NS_1614336837': 0.22092726826667786,
  'MF_NS_1614336858': 0.039660658687353134},
 'WS': {'MF_WS_1614336883': 0.03830385580658913,
  'MF_WS_1614336918': 0.042797017842531204,
  'MF_WS_1614336954': 0.03684352710843086},
 'EM': {'MF_EM_1614336991': 0.039454467594623566,
  'MF_EM_1614337035': 0.04111432284116745,
  'MF_EM_1614337065': 0.038048844784498215},
 'EE': {'MF_EE_1614337096': 0.04478411003947258,
  'MF_EE_1614337149': 0.04110008850693703,
  'MF_EE_1614337191': 0.053920503705739975},
 'LD': {'MF_LD_1614337214': 0.03826141357421875,
  'MF_LD_1614337243': 0.12023280560970306,
  'MF_LD_1614337265': 0.03942465782165527},
 'NEE': {'MF_NEE_1614337294': 0.0347551666200161,
  'MF_NEE_1614337336': 0.03617289289832115,
  'MF_NEE_1614337371': 0.030800309032201767},
 'NWE': {'MF_NWE_1614337414': 0.03048